# 4. 제너레이터 : 함수로 만들기

- PEP 255 Simple Generator

In [10]:
def make_gen():
    yield 1
    yield 2 
    yield 3

In [11]:
gen = make_gen()

In [12]:
gen

<generator object make_gen at 0x7fad98ae42d0>

In [13]:
from collections.abc import Iterator

In [14]:
isinstance(gen, Iterator)

True

In [15]:
next(gen), next(gen),next(gen)

(1, 2, 3)

In [16]:
next(gen)

StopIteration: 

In [17]:
def make_gen(n):
    for i in range(1, n+1):
        yield i

In [18]:
gen = make_gen(3)

In [19]:
next(gen), next(gen), next(gen)

(1, 2, 3)

In [20]:
def make_gen():
    print("1")
    yield 1
    print('2')
    yield 2
    print('3')
    yield 3
    print('4')

In [21]:
gen = make_gen() # 제너레이터 생성

In [22]:
next(gen)

1


1

In [23]:
next(gen)

2


2

In [24]:
next(gen)

3


3

In [25]:
next(gen)

4


StopIteration: 

# generator 대기하는 특성

yield가 하나 있는 함수

yield가 있는 부분을 특정 코드로 대체해서 실행시키기


예)

```
def some_generator():
    <code 1>
    yield
    <code 2>
```

+

```
<code A>
```

=

```
<code 1>
<code A>
<code 2>
```

+ with context

In [26]:
class GeneratorContextManager:
    def __init__(self, gen):
        self.gen = gen
        
    def __enter__(self):
        try:
            return next(self.gen)
        except StopIteration:
            raise RuntimeError
            
    def __exit__(self, error_type, error_value, traceback):
        if error_type is None:
            try:
                next(self.gen)
            except StopIteration:
                pass
            else:
                raise RuntimeError
        else:
            self.gen.throw(error_type, error_value, traceback)
            
            

In [27]:
def mycontextmanager(func):# < func 는 제너레이터 생성함수
    def inner(*args, **kwargs):
        return GeneratorContextManager(func(*args, **kwargs))
    return inner

In [28]:
@mycontextmanager
def tag(tag_name):
    print("<{}>".format(tag_name)) # <code 1>
    yield
    print("</{}>".format(tag_name)) # <code 2>

In [29]:
with tag('div'):
    print('Hello World!') # <code A>
    
# <code 1>
# <code A>
# <code 2>

<div>
Hello World!
</div>


In [30]:
@mycontextmanager
def print_a():
    try:
        print(a)
    except NameError:
        print("a는 없습니다.")
    yield
    print("a의 값은 : ", a)


In [31]:
with print_a():
    a = 12

a는 없습니다.
a의 값은 :  12


In [32]:
from contextlib import contextmanager

In [33]:
@contextmanager
def tag(tag_name):
    print("<{}>".format(tag_name)) # <code 1>
    yield
    print("</{}>".format(tag_name)) # <code 2>

In [34]:
with tag('div'):
    print('Hello World!') # <code A>

<div>
Hello World!
</div>


## 실습 : 제너레이터 

* fib 수열을 구하는 제너레이터

`1, 1, 2, 3, 5, 8, 13, ...`

In [35]:
def make_fib():
    a = 0
    b = 1
    while True:
        yield b
        a, b = b, a + b

In [36]:
fib = make_fib()

In [37]:
next(fib), next(fib), next(fib), next(fib), next(fib)

(1, 1, 2, 3, 5)

## 제너레이터의 단점

연속된 값들을 처리할때 제약

제너레이터는 slice 불가

In [38]:
def numbers():
    for i in range(10):
        yield i

In [39]:
gen = numbers()

In [40]:
list(gen)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [41]:
gen[:3]

TypeError: 'generator' object is not subscriptable

In [42]:
from itertools import islice

In [43]:
gen = numbers()

result = islice(gen, 3) 

In [44]:
list(result)

[0, 1, 2]

# 제너레이터는 chain, + 불가

In [45]:
def a():
    for i in range(3):
        yield i
        
gen = a()

In [46]:
gen + [3, 4, 5]

TypeError: unsupported operand type(s) for +: 'generator' and 'list'

In [47]:
gen + gen

TypeError: unsupported operand type(s) for +: 'generator' and 'generator'

In [48]:
from itertools import chain

In [49]:
gen = a()

it = chain(gen, [3, 4, 5])

In [50]:
list(it)

[0, 1, 2, 3, 4, 5]

# 제너레이터 (이터레이터)는 계속 반복 불가

In [51]:
def a():
    for i in range(3):
        yield i

In [52]:
gen = a()

In [53]:
next(gen), next(gen), next(gen)

(0, 1, 2)

In [54]:
next(gen), next(gen), next(gen)

StopIteration: 

In [55]:
from itertools import cycle

In [56]:
gen = a() # 0, 1, 2

In [57]:
it = cycle(gen) # 0, 1, 2, 0, 1, 2, 0, 1, 2, ...

In [58]:
next(it), next(it), next(it)

(0, 1, 2)

In [59]:
next(it), next(it), next(it)

(0, 1, 2)

In [60]:
import itertools

In [61]:
dir(itertools)

['__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_grouper',
 '_tee',
 '_tee_dataobject',
 'accumulate',
 'chain',
 'combinations',
 'combinations_with_replacement',
 'compress',
 'count',
 'cycle',
 'dropwhile',
 'filterfalse',
 'groupby',
 'islice',
 'permutations',
 'product',
 'repeat',
 'starmap',
 'takewhile',
 'tee',
 'zip_longest']

# yield

yield를 표현식으로 사용가능

In [62]:
def co():
    while True:
        x = yield 12
        print(x)

In [63]:
corou = co()

In [64]:
corou

<generator object co at 0x7fad98b7ee50>

In [65]:
next(corou)

12

In [66]:
corou.send(123)

123


12

# Generator

## send

## throw

## close

In [67]:
from collections.abc import Iterator, Generator

In [68]:
set(dir(Generator)) - set(dir(Iterator))

{'close', 'send', 'throw'}

# send()

In [207]:
help(Generator.send)

Help on function send in module collections.abc:

send(self, value)
    Send a value into the generator.
    Return next yielded value or raise StopIteration.



In [208]:
def gener():
    x = yield 1
    y = yield x+1
    z = yield y+1

In [214]:
gen = gener()

In [215]:
gen

<generator object gener at 0x10e0b64f8>

In [216]:
next(gen)

1

In [217]:
gen.send(2)

3

In [218]:
gen.send(3)

4

In [219]:
gen.send(4)

StopIteration: 

## 실습 : 코루틴 생성

1. 제너레이터 안에 보낸 값을 출력하고 StopIteration 발생하는 코루틴 (yield가 1개)
1. 무한반복되는 로직 -> 제너레이터가 특정 값을 넘겨 받아서 -> 총 합을 출력해주는 코루틴


In [220]:
def a():
    x = yield 
    print(x)

In [221]:
g = a()

In [222]:
# 첫번째 yield까지 가야하기때문에
next(g)

In [224]:
g.send(100)

StopIteration: 

In [225]:
def a():
    total = 0
    while True:
        x = yield
        total += x
        print(total)

In [226]:
g = a()

In [227]:
next(g)

In [228]:
g.send(100)

100


In [229]:
g.send(200)

300


In [230]:
g.send(300)

600


# throw

In [231]:
help(Generator.throw)

Help on function throw in module collections.abc:

throw(self, typ, val=None, tb=None)
    Raise an exception in the generator.
    Return next yielded value or raise StopIteration.



In [238]:
def a():
    total = 0
    while True:
        x = yield
        total += x
        print(total)

In [239]:
gen = a()
next(gen)

In [240]:
gen.throw(StopIteration)

RuntimeError: generator raised StopIteration

In [241]:
gen.send(12)

StopIteration: 

# close

In [242]:
help(Generator.close)

Help on function close in module collections.abc:

close(self)
    Raise GeneratorExit inside generator.



# yield from

PEP 380 

v 3.3

In [250]:
def subgenerator():
    for i in range(5):
        yield i
        
def maingen():
    for j in subgenerator():
        yield j

In [251]:
c = maingen()

In [252]:
list(c)

[0, 1, 2, 3, 4]

In [253]:
def subgenerator():
    yield from range(5)
        
def maingen():
    yield from subgenerator()

In [254]:
c = maingen()

In [255]:
list(c)

[0, 1, 2, 3, 4]

In [257]:
def maingen():
    yield from '1234'
    yield from (1, 2, 3, 4)
    
    
# def maingen():
#     for i in '1234':
#         yield i
#     for j in (1, 2, 3, 4):
#         yield j

In [258]:
gen = maingen()

In [259]:
list(gen)

['1', '2', '3', '4', 1, 2, 3, 4]

In [260]:
def sub():
    while True:
        data = yield
        print("출력 data :", data)
        
        
def main():
    yield from sub()

In [261]:
co = main()

In [262]:
next(co)

In [263]:
co.send(10)

출력 data : 10


In [266]:
def sub():
    while True:
        data = yield
        print("출력 data :", data)
        
        
def main():
    x = yield from sub()
    print(x, '를 받았습니다.')

In [267]:
co = main()
next(co)

In [269]:
co.send(2222)
co.send(123)

출력 data : 2222
출력 data : 123


In [270]:
def sub():
    while True:
        data = yield
        if data is None:
            break
        print("출력 data :", data)
        
        
def main():
    x = yield from sub()
    print(x, '를 받았습니다.')

In [271]:
co = main()
next(co)

In [272]:
co.send(333)

출력 data : 333


In [273]:
co.send(None)

None 를 받았습니다.


StopIteration: 

## return  in generator 

PEP 380

In [303]:
def sub():
    while True:
        data = yield
        if data is None:
            return 12
        print("출력 data :", data)
        
        
def main():
    x = yield from sub()
    print(x, '를 받았습니다.')

In [304]:
co = main()
next(co)

In [305]:
co.send(333)

출력 data : 333


In [306]:
co.send(None)
#next(co)

12 를 받았습니다.


StopIteration: 

### 리턴받는 형태

`return value`   
`raise StopIteration(value)`

## PEP 380

하위 제너레이터에게 위임하기 위한 문법 `yield from` , `return`



```
list[i]에 1부터 (i+1)까지의 합을 가지고 있는 리스트

[1, 3, 6, ...]
```

In [316]:
def sum_all():
    total = 0
    while True:
        data = yield
        if data is None:
            break
        total += data
    return total


def recode(storage):
    while True:
        value = yield from sum_all()
        storage.append(value)


def main(n):
    results = []
    for i in range(1, n+1):
        gen = recode(results)
        next(gen)
        for j in range(i+1):
            gen.send(j)
        gen.send(None)
    return results

In [317]:
main(20)

[1,
 3,
 6,
 10,
 15,
 21,
 28,
 36,
 45,
 55,
 66,
 78,
 91,
 105,
 120,
 136,
 153,
 171,
 190,
 210]

# 반복을 반복하는 형태

In [319]:
data = {
    'tom': [100, 80, 90, 30, 40],
    'john': [30, 50, 30, 70, 100],
    'emma': [80, 39, 69, 100, 90]
}

In [331]:
def average():
    total = 0
    count = 0
    while True:
        score = yield
        if score is None:
            break
        total += score
        count += 1
    return total / count

def recode(result, name):
    while True:
        result[name] = yield from average()
        
        
def main():
    result = {}
    for name in data:
        gen = recode(result, name)
        next(gen)
        for score in data[name]:
            gen.send(score)
        gen.send(None)
    return result

In [330]:
main()

{'tom': 68.0, 'john': 56.0, 'emma': 75.6}

In [332]:
from collections.abc import Coroutine

In [333]:
dir(Coroutine)

['__abstractmethods__',
 '__await__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_abc_impl',
 'close',
 'send',
 'throw']

In [334]:
!open .